## Decomposition Methods

In [27]:
from clifford import Cl
import sympy as sp
import cmath
import numpy as np

layout, blades = Cl(3)

def get_decomposition_tuples(matrix, approach=1):
    eigen_data = matrix.eigenvects()
    real_eigenpairs = []
    complex_structure = []

    processed = set()
    
    for eigenval, mult, vects in eigen_data:
        if not sp.im(eigenval):
            for vec in vects:
                vec_real = [float(val.evalf()) for val in vec]
                mv = sum(val * blade for val, blade in zip(vec_real, layout.basis_vectors_lst))
                real_eigenpairs.append((float(eigenval.evalf()), mv))
        else:
            # the if check below has issues (e.g. pi/2 rotation in e12 and e34 so we'll have i show up twice)
            if str(eigenval) in processed or str(sp.conjugate(eigenval)) in processed: 
                continue
            
            processed.add(str(eigenval))
            λ = complex(eigenval.evalf())
            c_k = abs(λ)
            θ_k = cmath.phase(λ)
            
            # Approach 1
            if approach == 1:
                vec = vects[0]
                vec_re = sp.re(vec).applyfunc(lambda x: float(sp.N(x)))
                vec_im = sp.im(vec).applyfunc(lambda x: float(sp.N(x)))

                vec_re_vals = [float(x) for x in vec_re]
                vec_im_vals = [float(x) for x in vec_im]

                mv_re = sum(val * blade for val, blade in zip(vec_re_vals, layout.basis_vectors_lst))
                mv_im = sum(val * blade for val, blade in zip(vec_im_vals, layout.basis_vectors_lst))

                # Issue when I reverse the order of the below wedge product
                B_k = mv_im ^ mv_re
            else:
                # Approach 2:
                v_complex     = mv_re + 1j * mv_im
                v_conjugate   = mv_re - 1j * mv_im

                # Compute bivector from imaginary part of v ^ v̄
                bivector_complex = v_complex ^ v_conjugate  # Complex-valued multivector
                imag_part = sum(mv.imag * blade for mv, blade in zip(bivector_complex.value, layout.blades_list))
                B_k = 0.5 * imag_part  # Real bivector representing the plane
            
            
            B_k_hat = B_k / (B_k | ~B_k)
            
            complex_structure.append((c_k, B_k, θ_k))

    return real_eigenpairs, complex_structure


def rotor(B, theta):
    """
    Construct rotor R = exp(- B̂ * theta / 2)
    where B̂ is the unit bivector (B / |B|).
    """
    B_sq_scalar = (B * B)[()]  # Get scalar (grade-0) part of B²
    if B_sq_scalar >= 0:
        raise ValueError("Bivector square is not negative. Cannot construct rotor.")
    B_norm = (-B_sq_scalar) ** 0.5
    B_hat = B / B_norm
    return (-B_hat * theta / 2).exp()


def decomposition(A):
    
    real_part, complex_part = get_decomposition_tuples(A)
    print(f"Real Part: {real_part}")
    print(f"Complex Part: {complex_part}")
    
    def operator(a):
        result = layout.MultiVector()
        
        # Real eigenvalue terms
        for λ_k, u_k in real_part:
            proj = (a << u_k) * ~u_k / (u_k | u_k)
            result += λ_k * proj
                    
        # Complex pair terms
        for c_k, B_k, θ_k in complex_part:
            R_k = rotor(B_k, θ_k)
            B_proj = (a << B_k) * ~B_k / (B_k | ~B_k)
            result += c_k * R_k * B_proj * ~R_k
        return result

    return operator


## Verify decomposition matches usual matrix vector product

### Define Linear Operator

In [29]:
θ = sp.pi / 2
R = sp.Matrix([
    [sp.cos(θ), -sp.sin(θ), 0],
    [sp.sin(θ),  sp.cos(θ), 0],
    [0,          0,         1]
])
A = 2 * R

A = sp.Matrix([
    [sp.cos(θ), -sp.sin(θ), 0],
    [sp.sin(θ),  sp.cos(θ), 0],
    [0,          0,         1]
])

B = sp.Matrix([
    [1, 0, 0],
    [0,  sp.cos(θ), -sp.sin(θ)],
    [0,          sp.sin(θ),         sp.cos(θ)]
])

C = A*B

### Decomposition

In [30]:
op = decomposition(C)

# Try on some input vector a
a = 1 * blades['e1'] + 2 * blades['e2'] + 3 * blades['e3']
output = op(a)
print("Output vector:", output)

Real Part: [(1.0, (1.0^e1) + (1.0^e2) + (1.0^e3))]
Complex Part: [(0.9999999999999999, -(0.86603^e12) + (0.86603^e13) - (0.86603^e23), -2.0943951023931957)]
Output vector: (3.0^e1) + (1.0^e2) + (2.0^e3)


### Matrix vector product

In [31]:
v = sp.Matrix([1, 2, 3])
# v = sp.Matrix([3, -1, 0])
Cv = C*v
Cv.evalf()

Matrix([
[3.0],
[1.0],
[2.0]])